# 1. 环境配置

## 1.1 python 环境准备

In [ ]:
! pip install openai==2.11.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/1a/39/47f9197bdd44df24d67ac8893641e16f386c984a0619ef2ee4c51fbbc019/beautifulsoup4-4.14.3-py3-none-any.whl (107 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/81/c4/34e93fe5f5429d7570ec1fa436f1986fb1f00c3e0f43a589fe2bbcd22c3f/pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/14/a0/bb38d3b76b8cae341dad93a2dd83ab7462e6dbcdd84d43f54ee60a8dc167/soupsieve-2.8-py3-none-any.whl (36 kB)
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11785 sha256=ddcdbf156dd708

## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

# 2. LangChain V1.0 ReAct Agent 搭建

## 2.1 简介

在最新的 LangChain V1.0 版本中，LangChain 官方已经全面将 Agent 组件中的相关内容接入到 LangGraph 中。LangGraph 是 LangChain 团队在 2024 年推出的新一代框架，用于构建具备“状态、记忆与多步推理能力”的智能体（Agent）。

它以“图（Graph）”为核心思想：把每个模型调用、工具执行、判断逻辑都视为一个节点，通过节点间的有向边来定义信息流动，从而让复杂的智能体流程（如 ReAct、对话管理、多智能体协作、人工中断等）都能用图结构清晰地表达。

相比旧版 LangChain 的线性链式调用，LangGraph 提供了可控、可追踪、可持久化的执行流，支持状态保存、检查点（Checkpoint）、时间回溯（Time Travel）等功能，是目前官方推荐的 Agent 开发核心框架。

## 2.2 LLM 模型

由于在 LangGraph 中，需要大模型支持工具调用（Function Calling）。因为通义千问明确在 LangChain 文档中表示支持，因此后续我们将使用 Qwen 系列模型进行演示。

In [ ]:
from langchain_community.chat_models import ChatTongyi
import os

llm = ChatTongyi(
  api_key=os.environ.get("DASHSCOPE_API_KEY"), 
  model="qwen-max")

response = llm.invoke("你好，请介绍一下你自己")

print(response.content)

你好！我是Qwen，一个由阿里云开发的预训练语言模型。我被设计用来生成各种类型的文本，如文章、故事、诗歌、故事等，并能够根据给定的上下文进行对话交流。我的目标是帮助用户提高创造力和生产力，同时提供准确有用的信息。无论是需要创意写作的帮助，还是想要了解某些知识，我都会尽力提供支持。很高兴认识你，希望我们能有愉快的交流！


## 2.3 Memory 记忆

在新版本里，我们不再需要通过 RunnableWithMessageHistory 的方式进行记忆的保留，在 LangGraph 下我们使用 InMemorySaver() 的方式进行保留。

所谓的短期记忆（Short-term Memory），实际上指的是系统仅在当前对话轮次中保存的临时上下文信息，用于维持一次连续的对话逻辑或局部推理过程。

在 LangGraph 中，InMemorySaver() 的作用就是在智能体（Agent）或图（Graph）执行的过程中，临时保存模型的输入输出、对话内容、工具调用记录以及节点状态等信息，从而使系统在一次会话（session）中能够“记住”先前的交互历史。

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver 
memory = InMemorySaver()

## 2.4 System Prompt 系统提示词

我们不需要再构建一大段的提示词了，而是只需要描述能力即可。

In [ ]:
system_prompt = "You are a helpful assistant"

## 2.5 Tool 工具

在工具的定义中，新版本并未作出太大的改变，我们还是可以使用之前工具定义的函数进行执行。

In [ ]:
from langchain.tools import tool

@tool
def calculate(what: str) -> str:
  """
  calculate:
  e.g. calculate: 4 * 7 / 3
  Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary
  """
  return str(eval(what))

@tool
def average_dog_weight(name: str) -> str:
  """
  average_dog_weight:
  e.g. average_dog_weight: Collie
  returns average weight of a dog when given the breed
  """
  name = name.lower()
  if "scottish terrier" in name:
    return "Scottish Terriers average 20 lbs"
  elif "border collie" in name:
    return "A Border Collie's average weight is 37 lbs"
  elif "toy poodle" in name:
    return "A Toy Poodle's average weight is 7 lbs"
  else:
    return "An average dog weighs 50 lbs"
  
tools = [calculate, average_dog_weight]

## 2.6 系统组装

在准备好了一些基础的组件以后，我们使用 create_agent 的方法将这些内容组合起来：

In [ ]:
from langchain.agents import create_agent

agent = create_agent(model=llm, 
           tools=tools, 
           system_prompt=system_prompt, 
           checkpointer=memory)

In [ ]:
result1 = agent.invoke({"messages": [{"role": "user", "content": "I have 2 dogs, a border collie and a scottish terrier. What is their combined weight? Could you double it?"}]}, config={"configurable": {"thread_id": "user_1"}})
print(result1)

{'messages': [HumanMessage(content='I have 2 dogs, a border collie and a scottish terrier. What is their combined weight? Could you double it?', additional_kwargs={}, response_metadata={}, id='7af138da-f6b1-4d27-89ea-565a349a52f8'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"name": "border collie"}', 'name': 'average_dog_weight'}, 'id': 'call_805875368a004fbba38d1a', 'index': 0, 'type': 'function'}, {'function': {'arguments': '{"name": "scottish terrier"}', 'name': 'average_dog_weight'}, 'id': 'call_34492eff073c453b82fccc', 'index': 1, 'type': 'function'}]}, response_metadata={'model_name': 'qwen-max', 'finish_reason': 'tool_calls', 'request_id': '02591330-c798-4ef8-b39a-b4d410e928b2', 'token_usage': {'input_tokens': 369, 'output_tokens': 47, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 416}}, id='lc_run--019b0678-74db-7ba2-bdb3-aa150f479b84-0', tool_calls=[{'name': 'average_dog_weight', 'args': {'name': 'border collie'}, 'id

假如只需要答案的话：

In [ ]:
print(result1["messages"][-1].content)

The calculation confirms that the combined weight of a Border Collie and a Scottish Terrier, when doubled, is 114 lbs.


# 3. 输出模式

## 3.1 流式输出

假如我们希望能够逐步看到内部的变化，而不是一次性全部打印，可以使用流式输出实现。

对于智能体而言，流式输出最大的价值并非仅仅只是一个个 token 的进行输出，更重要的是能够实现一段段动态的展示信息，从而能够更方便的检测智能体内部的变化。

在 LangGraph 中，可以有不同的流式输出模式，如：
- "values"：全局状态快照（展示整体的更新）
- "updates"：智能体执行进度（每一个节点都进行更新）
- "messages"：模型逐个 token 输出最终结果
- "custom"：自定义事件（比如“正在读取数据…”）

传入时可以通过 `for chunk in agent.stream(..., stream_mode="updates"):` 进行运行模式的设置。

### 3.1.1 value 模式

 values 模式下，流式输出的调用方式为：

In [ ]:
for step in agent.stream(
 {
  "messages": [{
   "role": "user",
   "content": "I have 2 dogs, a border collie and a scottish terrier. What is their combined weight? Could you double it?"
  }] # 提问的问题
 },
 config={
  "configurable": {"thread_id": "user_1"} # 短期记忆
 },
 stream_mode="values" # 可更换为 messages 或自定义方法
):
  step["messages"][-1].pretty_print()

================================ Human Message =================================

I have 2 dogs, a border collie and a scottish terrier. What is their combined weight? Could you double it?
================================== Ai Message ==================================
Tool Calls:
  average_dog_weight (call_fd7989e3057f41a1a3f225)
 Call ID: call_fd7989e3057f41a1a3f225
  Args:
    name: border collie
  average_dog_weight (call_4b3b77ad24e44ac6926592)
 Call ID: call_4b3b77ad24e44ac6926592
  Args:
    name: scottish terrier
================================= Tool Message =================================
Name: average_dog_weight

Scottish Terriers average 20 lbs
================================== Ai Message ==================================

The average weight of a Border Collie is 37 lbs, and the average weight of a Scottish Terrier is 20 lbs. Their combined average weight is 57 lbs. Doubling this combined weight gives us 114 lbs.

Let's confirm the calculation.
Tool Calls:
  calculate (

### 3.1.2 updates 模式

updates 模式下，流式输出的调用方式为：

In [ ]:
for chunk in agent.stream(
 {
  "messages": [{"role": "user",
   "content": "I have 2 dogs, a border collie and a scottish terrier. What is their combined weight? Could you double it?"
  }] 
 },
 config={"configurable": {"thread_id": "user_1"}},
 stream_mode="updates" 
):
  for step, data in chunk.items():
    print(f"step: {step}")
    print(f"content: {data['messages'][-1].content_blocks}")

step: model
content: [{'type': 'tool_call', 'id': 'call_aec720b57bc546b4919afe', 'name': 'average_dog_weight', 'args': {'name': 'border collie'}}, {'type': 'tool_call', 'id': 'call_93f73531a1fc47d2bf48a4', 'name': 'average_dog_weight', 'args': {'name': 'scottish terrier'}}]
step: tools
content: [{'type': 'text', 'text': "A Border Collie's average weight is 37 lbs"}]
step: tools
content: [{'type': 'text', 'text': 'Scottish Terriers average 20 lbs'}]
step: model
content: [{'type': 'text', 'text': 'The average weight of a Border Collie is 37 lbs, and the average weight of a Scottish Terrier is 20 lbs. Their combined average weight would be 57 lbs. When we double this combined weight, it would be 114 lbs.\n\nTo confirm, I will now calculate the doubled combined weight.'}, {'type': 'tool_call', 'id': 'call_f15a992045b34f00b11e40', 'name': 'calculate', 'args': {'what': '(37 + 20) * 2'}}]
step: tools
content: [{'type': 'text', 'text': '114'}]
step: model
content: [{'type': 'text', 'text': 'Th

### 3.1.3 messages 模式

In [ ]:
for token, metadata in agent.stream(
 {
  "messages": [{
   "role": "user",
   "content": "I have 2 dogs, a border collie and a scottish terrier. What is their combined weight? Could you double it?"
  }]
 },
 config={
  "configurable": {"thread_id": "user_1"},
 },
 stream_mode="messages" 
):
  print(f"{token.content}", end=" ")    # 结尾添加一个空格可以更明显看到流式输出的形式

 A Border Collie's average weight is 37 lbs Scottish Terriers average 20 lbs The average weight of a Border Collie is 37 lbs, and the average weight of a Scottish Terrier is 20 lbs. Their combined average weight is 57 lbs. When we double this combined weight, it would be 114 lbs.

Let's confirm the calculation by using the `calculate` function. 114 The calculation confirms that the combined weight of a Border Collie and a Scottish Terrier, when doubled, is 114 lbs. 

## 3.2 结构化输出

在新版的 create_agent 中，我们还可以添加结构化输出的组件，使得最终返回的结果不再是字符串，而是一个 JSON 格式的内容。

和之前 LangChain 的格式化输出一样，我们需要先通过 Pydantic 定义一个基本的格式：

In [ ]:
from pydantic import BaseModel, Field

class DogWeightSummary(BaseModel):
    """Structured summary of dog weight analysis."""
    dogs: list[str] = Field(description="List of dog breeds mentioned")
    combined_weight: float = Field(description="Total combined weight of the dogs in pounds")
    doubled_weight: float = Field(description="Double of the total combined weight")

这里我们就定义了要结构化返回的内容是列表格式的 dogs，浮点数格式的 combined_weight 和 doubled_weight ，并且都对应的添加了解释。

这个时候我们把这部分的内容载入到 create_agent 中：

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt,
    checkpointer=memory,
    response_format=DogWeightSummary)

然后我们同样需要设置 thread_id 并将问题进行传入：

In [ ]:
result1 = agent.invoke({"messages": [{"role": "user", "content": "I have 2 dogs, a border collie and a scottish terrier. What is their combined weight? Could you double it?"}]}, config={"configurable": {"thread_id": "user_1"}})
print(result1)

ValueError: request_id: 5940644d-f614-44bf-b163-ba52a7d24ea1 
 status_code: 400 
 code: InvalidParameter 
 message: <400> InternalError.Algo.InvalidParameter: tool_choice is one of the strings that should be ["none", "auto"]

但是很可惜的是 ChatTongyi 并不支持 LangChain 的 Agent 结构化输出。假如想要体验的话，需要使用 OpenAI 或者 Anthropic 系列的模型：

In [ ]:
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-5-nano")

from langchain.agents import create_agent
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt,
    checkpointer=memory,
    response_format=DogWeightSummary)

result1 = agent.invoke({"messages": [{"role": "user", "content": "I have 2 dogs, a border collie and a scottish terrier. What is their combined weight? Could you double it?"}]}, config={"configurable": {"thread_id": "user_1"}})
print(result1)